In [5]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))


article_text = """
0,7, بسم الله الرحمن الرحيم ، أهلا بحضرتكم في فيديو خاص تعليقا على النصر المظفر والمبين
7,14, اللي حققه الأستاذ خالد البلشي في نقابة الصحفيين وفوزه بمنصب نقيب الصحفيين
14,24, تقدر تقول كأول نقيب مستقل تماما ، يعني خارج من حتى ما كان يعرف بمؤسسات الجنوب الصحفي
24,35, هي مؤسسات الفقيرة زي روز اليوسف ودار الهلال اللي كان بيطلع منها نقباء يعني بعضهم مدعوم حكوميا وبعضهم غير ذلك أو مستقلين
35,44, لكن هنا بتتكلم عن نقيب أولا محسوب بشكل واضح على قوى المعارضة ، على اليسار ، على الثورة يناير
44,52, بشكل لا يدع مجال للشك ، بتتكلم عن نقيب صحفيين رئيس تحرير ليه مش بس موقع محجوب
52,62, اللي هو موقع درب وإنما لعدة مواقع صحفية محجوبة متتالية بدأها بموقع كاتب ثم موقع البداية ثم موقع درب
62,73, وكان في حالة من المطاردة بينه وبين السلطات الحاجبة لدرجة أنه أسرع مواقع كنت تتحجب هي اللي لها علاقة بخالد البلشي
74,80, فأنت عندك هنا حدث ، لو في الأيام العادية يبقى حدث كبير
80,89, اللي هو الأيام الاستبداد المباركي ناعم المخالب اللي بيدير النقابات بالتليفون وبشكل يعني
89,97, فيه نوع من أنواع المرونة والسلاسة فما بالك بهذه الأيام اللي الحكاية فيها يعني حفرة على الناشف
97,106, وفيها غشومية في إدارة العمل السياسي والنقابي مما يعطي لهذا الفوز دلالاته كبيرة جدا
106,120, وخصوصا أنه مش فوز فردي وإنما الفوز مجموعة أسماء وانهزام بعضه يعني فاضح وبعضه عادي لمجموعة أسماء محسوبة على النظام
120,133, ولا تجد أي حرج في ذلك يعني وفوز كمان في مواجهة مال يعني يصل إلى فكرة أنه يبقى فيه يتندر الصحفين على وجود كاراتين لبشارة
133,143, طبعاً ده كان يوم الكاراتين العظيم عبد الفتح السيسي في الاستاد بيوزع كاراتين ورجالته في نقابة الصحفين بيوزعوا كاراتين مشويات
143,154, متخيلين أنه ده هيحسم موضوع نقابة الصحفين طبعاً مشهد الاستاد ده احنا تكلمنا عنه قبل كده اللي هو في إطار احتقار المصريين ومش هأكرر الحديث عنه
154,163, ويعني يبدو أنه ما فيش أي حد عاقل حتى في النظام مستعد أنه ينبه إلى خطورة المشاهد
163,176, أنا مبارحة كنت بتفرج على قناة من القنوات التلفزيونية الحكومية كلها طبعاً قنوات حكومية مشاهد مرعبة في انحططها واحتقارها للناس
176,183, وأنك تظهر زباط شرطة هم بيوزعوا كاراتين على الناس وجايبهم بوشهم وبتمنى عليهم
183,192, ما فيش حد بيفكر لأ خلاص هو الموضوع يعني ايدك والأرض ما فيش حد حتى مدرك خطورة داع لشكل البلد
192,207, أو أنه بلد يعني بتحتفل بخبتها ومش قادرة حتى تدير المشهد المزري بتاع الفقر بشكل يعني فيه حفظ شوية لكرامة الناس
208,212, فانت بتتكلم في نقابة الصحفيين اللي هي نقابة من نقابات الرأي
212,218, وبتتكلم عن مشهد تقدر تضعه في صورة أكبر اللي هو ما حدث في نقابة الأطباء
218,225, برضو النقيم فيها مش محسوب على النظام معبر عن إرادة الجمعية العمومية للنقابة
225,230, نقابة المهندسين نفس الحكاية المناوشات الكتيرة اللي حصلت في نقابة المحامين
230,233, واللي بتنبيه انه فيه مشهد وفيه صورة مختلفة
233,241, طبعا في أوقات زي دي وفي أوضاع زي دي نفتكر بيت الشاعر الكبير فؤاد حداد عمالة بالغ في الأمل واليأس
241,249, وانه تلاقي ناس يعني بتشوف أداءهم عندك على الـ timeline في حساباتك على الـ social media
249,255, تشوف أصدقاء قبل الانتخابات بيوم يقولك يا عم انسى يا عم انتوا مجنين يا عم خالد البلش مين
255,258, الراجل ده واضح انه هو ايه يعني مسدق نفسه
258,264, والراجل ده طيب ولا حاجة مش حد يعقله يا جماعة من صحابه ويقوله ايه اللي انت بتعمله
264,269, وتاني يوم ايه ده ده بقى انه بيعملها ده بقى انه هيعملها ده بقى انه عملها
269,274, وبعدين بعد شوية تلاقي نفس الشخص يقولك ايه امه اكيد مطبخة ما مش معقولة
274,278, ما هو لازم يسيبه حاجة عشان يفكه الضغط على النظام ويبدأ بقى ايه
278,290, يحاول يبرر يأسه الشديد بانه مش قادر يعني ينظر الى فكرة ان فيه واحد نزل الشارع الصحفي
290,298, ولف من مؤسسة لمؤسسة وتحمل غلسات وغتتات في بعض الاماكن زي ما حصل له في وكالة انباء الشرق الاوسط
298,308, وتحمل سماع الشكوى وتحمل وده جاي طبعا من خبرة في العمل النقابي بقالها سنين
308,315, وادرك انه في لحظة ممكن يتم تغيير فيها شيء
315,321, فكون بقى ان في حد بيبالغ في الامل وياخد بقى هذا المشهد
321,330, اللي هو جاي من تضحيات وشغل وفريق عمل متكامل مش بس خالد البلشي وانما مجموعة ضخمة حواليه
330,335, سواء من الصحفيين المخضرمين او من شباب الصحفيين صعبان عليهم حال مهنتهم
335,341, رفضين ان هم الوضع يستمر بهذا الشكل المزري وقرروا ان هم يغيروا اوضاعهم
341,352, فكون ان انت تاخد المشهد ده وعايز تبالغ في الامل وتصوره الى انه ممكن يتكرر فيه انتخابات رئاسية قادمة في السنة الجاية ده لوقت عملت يعني
352,362, وتطلع منه بعضات وعبر انه الناس لما يقرروا ان هم يتغيروا فممكن يغيروا الواقع بحقك تماما
362,375, لكن طول الوقت انت محتاج تفتكر انه لعبة الانتخابات الرئاسية دي او قوانينها لعبة مختلفة تماما بيحكمها عوامل كثيرة جدا منها عوامل اقليمية منها عوامل دولية
375,383, منها كمان التصدعات اللي داخل المؤسسة الحاكمة وبالتحديد المؤسسة العسكرية والاجهزة السيادية زي المخابرات او ما يطلق عليه
383,392, الاجهزة السيادية او الاستعبادية لكن برضو يظل انه الشعب المصري لو عايز حاجة يعملها هيعملها سواء في الشارع او في صندوق الانتخابات
392,400, مش لانه في شيء خاص او مختلف ولكن دا عملته شعوب كثيرة جدا والامثلة في ذلك اكثر من ان يعني نذكرها الآن
400,413, اكفينا ما حدث مثلا في شيلي في ضل قبضة حاكم يعني عتيد وجبار زي بنوشي بمؤسسته العسكرية الداعمة وجي الاستفتاء اللي هو بتاع نوق قلب الموازين
413,425, فطول الوقت فيه معاملات كثيرة جدا وعوامل كثيرة جدا تخلينا نحكم على التجربة لكن المؤكد انه اللي حصل دا في نقابة صحفيين
425,435, شيء مختلف شيء له ما بعده شيء يخلي يبقى فيه جرس انذار بالنسبة للنظام الحاكم في تعامله مع الشارع
435,448, وانا اعتقد ومش معلومات انما تصور انه فيه طاقم بدأ يفرغ نفسه دلوقتي لافشال اي محاولة للاتفاق على مرشح رئاسي في الانتخابات الرئاسية القادمة
448,465, سواء تمت في معدها او مش في معدها بس في دلوقتي جرس انذار الى انه ايه دا ممكن يحصل اتفاق بشكل ما يستغل حاجات احنا كنا متخيلين انها متحصلش فلازم يتم بقى ايه القضاء عليها من المنبع
465,476, وانا اعتقد انه هيبقى وده بقى جزء فيه مش بس تصوراته انما معلومات انه فيه جزء كبير جدا من التحقيق عن ايه اللي حصل وليه دا حصل
476,488, وليه تم السماح بفوز خالد البلشي وليه تم الاصرار على اختيار واحد زي خالد ميري المرشح له رئيس تحرير الاخبار
488,507, وانه يكون بالمناسبة خالد ميري وخالد البلشي دفعة واحدة في كلية اعلام وهم الدفعة الابلي مباشرة فانا اعرفهم كويس بل يمكن حتى معرفتي بخالد بيري كانت اسبق في كلية اعلام اكتر من خالد البلشي اللي عرفته بعد كده في روزاليوسف ثم في الدستور
508,524, طبعا موضوع برضو اختيار ميري يعني انه اللي بقى المرشح الميري على قولة اللي قال يعني انه مرشح المدني هزم المرشح الميري زي ما كتب الصديق احمد الفخراني
524,540, لكن الفكرة هنا لانه خالد ميري مش جديد على النقابة هو امين صندوق وكان رئيس لجنة اظن خدمات اسكان او حاجة قبل كده يعني اشتغل في العمل النقابي من سنين فهذا مش اختيار جديد
540,562, لكن ما كانش هو المرشح المدعوم من ما يطلق عليها مؤسسات الشمال او المؤسسات الحكومية اللي هي نظام مبارك فخخ فيها النقابة من سنين طويلة وحط فيها بقى ايه موظفين اداريين خدوا عضوية نقابة بل ووصل الامر الى سوائين مع الاحترام طبعا لمهنة السواء او المهنة الادارية
562,589, لكن نقابة صحفيين دي مهنة ليها علاقة بمهنة الصحافة نفسها باللي بيمرسوا العمل الصحفي بتفريعاته وبتنوعاته والكلام في ذلك اشرحه يطول ولعب فيه بقى العتولة الرؤساء المجالس الادارات العتيدين اللي كل واحد فيهم اعدله زي مبارك برضو حاجة وعشرين سنة عندك ابراهيم نافع عندك ابراهيم سعدة جلالد ودر سامير رجب مكرم محمد احمد
589,618, كل واحد في منطقته وفي مسحته فكانت السيطرة قوية المرة دي الحجد معرفوش يعملوه لانه اختاروا اسم مش محل اجماع واسم فيه تصدعات بخصوصه ومن هنا جاية فكرة الغشومية في الاختيار يعني كان عندهم فرصة مثلا ان هم يختاروا واحد زي مثلا عبد المحسن سلام من الاهرام او اسماء كتيرة ممكن تحقق الشكل اللي هو اقرب الى الشكل المباركي
618,639, اللي كان بيلعب اللعبة النقبية دي من خلال انه ينزل بحد مثلا زي ابراهيم نافع فيقول للصحفيين ان كل اللي انتو هتطلبوه من مطالب مادية هتجاب لان الراجل ده ودنه ودن مبارك في بؤه يعني فيقدر يوصل بقى ايه الطلبات بتاعتك وينتزعلك وحقوك
640,666, وانتو ايه الواحد زي مكرم محمد احمد اللي كان بيشارك في كتابة خطب مبارك فبالتالي هيقدر يلبلكوا طلباتكوا بس النظام ما ينزلش سامير رجب حتى لما مرة سامير رجب قرر انه ينزل الانتخابات كانت حوسها ويعني طرد اش قرر طرد من ده ونزله ابراهيم نافع لانهم معرفين ان في حدود فيه تقدر تلعب بفكرة ناس عندهم الحد الادنى من المهنية
666,694, اللي تخليهم حتى مثلا لما يجي الحكومة تطلع قانون زي قانون رقم 93 لسنة 95 اللي هو لتكميم حرية الصحافة فيهنبري ابراهيم نافع وباسم النقابة للدفاع عن حرية الصحافة ويحصل حشد ويقف مع الصحفيين لانه فيه حدود معينة للمخطرة بفكرة مهنة الصحافة
694,708, طبعا ذلك عهد مضى وولى وكانت الحكومة فيه قدرة من خلال وجود اسماء معينة في الأمن الوطني ومكانت المخابرات العامة مش ضليعة قوي في العمل إدها مش في العمل الداخلي
708,732, وفي جوة الأمن الوطني فيه اسماء مختلفة كل واحد فيهم يمسك ملف ويديره مع اللي موجودين فيه وحوالين حسن مبارك فيه اسماء عندها خبرة بالعمل السياسي فكانت الحكاية بيبقى فيها نوع من أنواع يعني إما أنه هم يفتحوا خطوط مع شخصيات مستقلة زي مثلا حمد السيد في انقابة الأطباء وغيره يعني
732,756, أو يسمحوا مثلا بأنه يبقى فيه شخصية زي النقيب بأشهر نقابة المحامين أحمد الخواجة أو رجاء عطية أو سامح عشور ويبقى فيه لعب من خلال الشخصيات دي اللي هي شخصيات مستقلة وعامة ويبقى الضغط عليها من تحت لتحت مش بالشكل الفج اللي بقى موجود في عهد عبد الفتح السيسي
756,776, وجزء العساكر اللي هم حوالي يعني حتى العساكر فيهم اللي لابسين مدني يعني اللي هي نفس عقلية الحفر على الناشف واللي هو ايه بقى وإن كان عجيبكوا بقى وإيه وإنتوا فاكرين نفسكوا وإيه وعبد الناصر محظوظ بإعلامه وطلعوا من فوقنا بقى والبلد مستهدفة والحاجات اللي ايه
776,799, الافكار البلهاء والمنحطة ويمكن ييجي هنا نموذج انه حتى لما يقرروا وده لعب دور في الانتخابات الاخيرة انه هم اختاروا عينات من عينات الدبة اللي فشخت صاحبها زي مثلا فيه واحد اسمه احمد الطاخري ده رئيس تحرير روزاليوسف وممثلسكين والقاهرة الاقبارية
800,826, وحاجة من النوع اللي هو ينطبق عليه قول الشاعر خريان مدريان ازاي خريان مدريان ده مش افتقى تعليه انك تلاقي واحد يوصل الى هذه المناصب وليس في تاريخه مقال واحد مش هقولك خبر او حملة صحفية او تحقيق وانما كل المسألة علاقاته بيه الزباط وفلان بيه وعلي بيه وبتاعه والناس اللي ايه بيسندوه وبيدعموه
"""

model_name = "csebuetnlp/mT5_m2o_arabic_crossSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    # [article_text],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4,
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)


فاز الأستاذ خالد البلشي بمنصب نقيب الصحفيين في مجلس النواب المصري ليصبح أول صحفي مستقل تماما ويشغل هذا المنصب.
